In [103]:
import pandas as pd
import numpy as np
import re # Thư viện xử lý biểu thức chính quy (Regex) cho văn bản
import os


import sys
sys.path.append(os.path.abspath(r"..\src"))

import config as cf
import data_processing as dp 
import utils as ut 

In [104]:

# Tạo thư mục processed nếu chưa có
os.makedirs(os.path.dirname(cf.PROCESSED_DATA), exist_ok=True)

df = pd.read_csv(cf.RAW_DATA)
print(f"Dữ liệu gốc có kích thước: {df.shape}")
df.head(3)

Dữ liệu gốc có kích thước: (11033, 31)


,source_name,source_link,event_id,event_date,event_time,event_title,event_description,location_description,location_accuracy,landslide_category,...,country_code,admin_division_name,admin_division_population,gazeteer_closest_point,gazeteer_distance,submitted_date,created_date,last_edited_date,longitude,latitude
0,AGU,https://blogs.agu.org/landslideblog/2008/10/14...,684,08/01/2008 12:00:00 AM,NaN,"Sigou Village, Loufan County, Shanxi Province","occurred early in morning, 11 villagers buried...","Sigou Village, Loufan County, Shanxi Province",unknown,landslide,...,CN,Shaanxi,0.0,Jingyang,41.02145,04/01/2014 12:00:00 AM,11/20/2017 03:17:00 PM,02/15/2018 03:51:00 PM,107.4500,32.5625
1,Oregonian,http://www.oregonlive.com/news/index.ssf/2009/...,956,01/02/2009 02:00:00 AM,NaN,"Lake Oswego, Oregon",Hours of heavy rain are to blame for an overni...,"Lake Oswego, Oregon",5km,mudslide,...,US,Oregon,36619.0,Lake Oswego,0.60342,04/01/2014 12:00:00 AM,11/20/2017 03:17:00 PM,02/15/2018 03:51:00 PM,-122.6630,45.4200
2,CBS News,https://www.cbsnews.com/news/dozens-missing-af...,973,01/19/2007 12:00:00 AM,NaN,"San Ramon district, 195 miles northeast of the...",(CBS/AP) At least 10 people died and as many a...,"San Ramon district, 195 miles northeast of the...",10km,landslide,...,PE,Junín,14708.0,San Ramón,0.85548,04/01/2014 12:00:00 AM,11/20/2017 03:17:00 PM,02/15/2018 03:51:00 PM,-75.3587,-11.1295


In [105]:
# Danh sách các cột cần xóa
cols_to_drop = [
    'event_id',
    'event_import_id', 
    'event_import_source', 
    'created_date', 
    'submitted_date', 
    'last_edited_date',
    'source_link', 
    'photo_link', 
    'storm_name',
    'country_code', 
    'event_time', 
    'notes',
    'gazeteer_closest_point', 
    'gazeteer_distance'
]

# Xóa cột nếu nó tồn tại trong dataframe
df.drop(columns=[c for c in cols_to_drop if c in df.columns], inplace=True)

print(f"Kích thước sau khi xóa cột thừa: {df.shape}")


Kích thước sau khi xóa cột thừa: (11033, 18)


In [106]:
print("Đang sửa lỗi hiển thị văn bản (Font encoding)...")


cols_to_fix = ['event_description', 'location_description', 
               'admin_division_name', 'event_title','source_name', 'admin_division_name', 'country_name']
for col in cols_to_fix:
    df[col] = df[col].apply(ut.fix_encoding)

print("Đã sửa xong lỗi font cho các cột văn bản.")

Đang sửa lỗi hiển thị văn bản (Font encoding)...
Đã sửa xong lỗi font cho các cột văn bản.


In [107]:
# Chuyển đổi sang datetime, các giá trị lỗi sẽ biến thành NaT
df['event_date'] = pd.to_datetime(df['event_date'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

print("Đã hoàn thành xử lý thời gian.")

Đã hoàn thành xử lý thời gian.


In [108]:
# === Xử lý dữ liệu văn bản bị thiếu (Text Imputation) ===

# 1. Xử lý 'event_description'
df['event_description'] = df['event_description'].fillna(df['event_title']).fillna('unknown')

# 2. Xử lý 'location_description'
df['location_description'] = df['location_description'].fillna(df['admin_division_name']).fillna('unknown')

# 3. Đảm bảo tất cả đều là chuỗi (string) để tránh lỗi khi chạy hàm clean_text
text_cols = ['event_description', 'location_description', 'event_title']
for col in text_cols:
    df[col] = df[col].astype(str)

print(f"Đã xử lý xong dữ liệu văn bản thiếu.")
print(f"Số lượng missing còn lại: {df[text_cols].isna().sum().sum()}")

Đã xử lý xong dữ liệu văn bản thiếu.
Số lượng missing còn lại: 0


In [109]:
# extraction_result = df['event_description'].apply(lambda x: dp.extract_casualties(x))
# Tạo danh sách descriptions
descriptions = df['event_description'].astype(str).tolist()

# Dùng list comprehension - nhanh hơn apply
extraction_result = [dp.extract_casualties(desc) for desc in descriptions]
extraction_result = pd.Series(extraction_result, index=df.index)

extracted_fatalities = pd.to_numeric(extraction_result.apply(lambda x: x[0] if isinstance(x, tuple) and len(x) > 0 else None), errors='coerce')
extracted_injuries = pd.to_numeric(extraction_result.apply(lambda x: x[1] if isinstance(x, tuple) and len(x) > 1 else None), errors='coerce')


# Đảm bảo cột gốc cũng là số để so sánh
df['fatality_count'] = pd.to_numeric(df['fatality_count'], errors='coerce')
df['injury_count'] = pd.to_numeric(df['injury_count'], errors='coerce')

# --- 3. Xử lý FATALITY_COUNT ---
update_mask_fat = extracted_fatalities.notna() & (
    df['fatality_count'].isna())


df.loc[update_mask_fat, 'fatality_count'] = extracted_fatalities
df.loc[update_mask_fat, 'fatality_imputed'] = True


# --- 4. Xử lý INJURY_COUNT  ---
update_mask_inj = extracted_injuries.notna() & (
    df['injury_count'].isna() )


df.loc[update_mask_inj, 'injury_count'] = extracted_injuries
df.loc[update_mask_inj, 'injury_imputed'] = True


# Hoàn tất việc tạo cột cờ (Những dòng còn lại sẽ là NaN, ta fill False)
df['fatality_imputed'] = df['fatality_imputed'].fillna(False).astype(bool)
df['injury_imputed'] = df['injury_imputed'].fillna(False).astype(bool)

# --- 5. Hoàn tất: Điền 0 MẶC ĐỊNH cho những ô vẫn còn thiếu ---
df['fatality_count'] = df['fatality_count'].fillna(0).astype(int)
df['injury_count'] = df['injury_count'].fillna(0).astype(int)


# --- 6. Kiểm tra kết quả ---
num_updated_fatality = df['fatality_imputed'].sum()
num_updated_injury = df['injury_imputed'].sum()

print(f"\n[Kết quả Smart Imputation & Correction]")
print(f"Đã tự động cập nhật/điền {num_updated_fatality} dòng số người chết.")
print(f"Đã tự động cập nhật/điền {num_updated_injury} dòng số người bị thương.")


[Kết quả Smart Imputation & Correction]
Đã tự động cập nhật/điền 62 dòng số người chết.
Đã tự động cập nhật/điền 335 dòng số người bị thương.


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13976\3155295027.py:36: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['fatality_imputed'] = df['fatality_imputed'].fillna(False).astype(bool)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13976\3155295027.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['injury_imputed'] = df['injury_imputed'].fillna(False).astype(bool)


In [110]:
# 1. Chuẩn hóa các cột phân loại chính
# Chuyển về chữ thường (lowercase) và xóa khoảng trắng thừa (strip)
categorical_features = ['landslide_category', 'landslide_trigger', 'landslide_size', 'country_name', 'landslide_setting']


size_mapping = {
    'small': 'small',
    'medium': 'medium',
    'large': 'large',
    'very_large': 'very_large',
    'catastrophic': 'very_large', # Gộp catastrophic vào very_large
    'unknown': 'unknown',
    'nan': 'unknown'
}
# Chỉ map những giá trị có trong từ điển, còn lại giữ nguyên hoặc gán unknown
df['landslide_size'] = df['landslide_size'].map(size_mapping).fillna('unknown')
landslide_counts = df['landslide_size'].value_counts().sort_values(ascending=False)

print("Đã chuẩn hóa dữ liệu văn bản.")
print(df['landslide_size'].value_counts())

Đã chuẩn hóa dữ liệu văn bản.
landslide_size
medium        6551
small         2767
unknown        860
large          750
very_large     105
Name: count, dtype: int64


In [111]:
# --- Xử lý Admin Division Population (Thay thế số 0 và NaN) ---

# Bước 1: Thay số 0 bằng NaN để dễ xử lý chung với các ô trống
df['admin_division_population'] = df['admin_division_population'].replace(0, np.nan)

# Bước 2: Điền NaN bằng giá trị trung vị (median) của TỪNG QUỐC GIA
# Nếu quốc gia đó cũng toàn NaN, thì mới dùng trung vị toàn cầu
global_median = df['admin_division_population'].median()

df['admin_division_population'] = df.groupby('country_name')['admin_division_population'].transform(
    lambda x: x.fillna(x.median())
)

# Bước 3: Điền nốt những ô vẫn còn NaN (do cả quốc gia đó thiếu số liệu) bằng global median
df['admin_division_population'] = df['admin_division_population'].fillna(global_median)

# Đảm bảo kiểu dữ liệu là float hoặc int
df['admin_division_population'] = df['admin_division_population'].astype(float)

print(f"Đã xử lý dân số = 0 và NaN bằng Median theo quốc gia.")

Đã xử lý dân số = 0 và NaN bằng Median theo quốc gia.


d:\Study\HCMUS\2025-2026_HK1\Lập trình cho KHDL\Final Project\Global-Landslide-Catalog-NASA-Analysis\vevn\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
d:\Study\HCMUS\2025-2026_HK1\Lập trình cho KHDL\Final Project\Global-Landslide-Catalog-NASA-Analysis\vevn\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [112]:
# Danh sách các cột phân loại cần đảm bảo không có giá trị thiếu
cols_to_fill_unknown = [
    'country_name', 
    'admin_division_name',
    'location_description',
    'landslide_trigger',
    'landslide_setting',
    'landslide_category',
    'event_description',
    'event_title'
]

print("Đang xử lý dứt điểm các giá trị thiếu cho các cột địa lý/phân loại...")

for col in cols_to_fill_unknown:
    # Bước 1: Đảm bảo cột là kiểu string/object để chứa 'unknown'
    df[col] = df[col].astype(str)
    
    # Bước 2: Điền NaN thực sự (nếu còn) bằng chuỗi rỗng
    df[col] = df[col].fillna('')
    
    # Bước 3: Thay thế chuỗi rỗng "" (từ các bước làm sạch) bằng 'unknown'
    df.loc[df[col] == '', col] = 'unknown'
    
    # Bước 4 (Kiểm tra): Đảm bảo các giá trị 'nan' (chuỗi) cũng được chuẩn hóa
    df.loc[df[col].str.lower().str.strip() == 'nan', col] = 'unknown'

    df[col] = df[col].apply(dp.clean_text)

print("Đã hoàn thành điền thiếu 'unknown' cho tất cả các cột phân loại chính.")

# Kiểm tra cuối cùng
print(df[cols_to_fill_unknown].isna().sum())

Đang xử lý dứt điểm các giá trị thiếu cho các cột địa lý/phân loại...
Đã hoàn thành điền thiếu 'unknown' cho tất cả các cột phân loại chính.
country_name            0
admin_division_name     0
location_description    0
landslide_trigger       0
landslide_setting       0
landslide_category      0
event_description       0
event_title             0
dtype: int64


In [113]:
# Lọc chỉ lấy các dòng có tọa độ hợp lệ
df = df[
    (df['latitude'] >= -90) & (df['latitude'] <= 90) &
    (df['longitude'] >= -180) & (df['longitude'] <= 180)
]

print(f"Kích thước sau khi lọc tọa độ: {df.shape}")

Kích thước sau khi lọc tọa độ: (11033, 20)


In [114]:
# Lưu file
df.to_csv(cf.PROCESSED_DATA, index=False, encoding='utf-8-sig')
print(f"Đã lưu dữ liệu sạch tại: {cf.PROCESSED_DATA}")

Đã lưu dữ liệu sạch tại: ..\data\processed\Global_Landslide_Processed.csv
